# Important: Add competition dataset as input dataset first for this notebook to work

In [3]:
!pip install -U bitsandbytes peft accelerate datasets sentencepiece wandb python-dotenv wtpsplit langchain
!pip install flash-attn --no-build-isolation
!pip install wtpsplit==2.1.1
!pip install syntok==1.4.4
!pip install omegaconf
!pip install wandb
!pip install --upgrade transformers trl
!pip install pandas numpy

  Using cached transformers-4.45.2-py3-none-any.whl.metadata (44 kB)
Using cached transformers-4.45.2-py3-none-any.whl (9.9 MB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.3
    Uninstalling transformers-4.46.3:
      Successfully uninstalled transformers-4.46.3
  Using cached flash_attn-2.7.2.post1.tar.gz (3.1 MB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [20 lines of output]
      fatal: not a git repository (or any of the parent directories): .git
      /tmp/pip-install-s3c6iu8t/flash-attn_8e290eda09634045940638160da47045/setup.py:99: UserWarning: flash_attn was requested, but nvcc was not found.  Are you sure your environment has nvcc available?  If you're installing within a container from https://hub.docker.com/r/pytorch/pytorch, only images whose names contain 'devel' will provide nvcc.
        warnings.warn(
      Tra

In [2]:
!git clone https://github.com/Reennon/gen-ai-nlp-lab-1

Cloning into 'gen-ai-nlp-lab-1'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 23 (delta 4), reused 20 (delta 4), pack-reused 0 (from 0)
Unpacking objects: 100% (23/23), 173.84 KiB | 3.86 MiB/s, done.


In [10]:
%cd gen-ai-nlp-lab-1
!ls

/kaggle/working/gen-ai-nlp-lab-1
README.md  notebooks  poetry.lock  pyproject.toml  src


In [11]:
import os
import pandas as pd

from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig
from huggingface_hub import login
from transformers import PreTrainedTokenizerBase, BitsAndBytesConfig
from tqdm import tqdm
from torch.utils.data import Dataset
from datasets import Dataset
from src.prompts.prompts import (NERPrompt1, NERPrompt2, NERPrompt3, NERPrompt4)
from src.prompts.examples import (
    ELEVEN_SHOT_EXAMPLES_DICT, 
    FIVE_SHOT_EXAMPLES_DICT, 
    THREE_SHOT_EXAMPLES_DICT, 
    ZERO_SHOT_EXAMPLES_DICT
)


from kaggle_secrets import UserSecretsClient

In [5]:
QUANTIZE_4BIT = True
device   = "cuda:0"

In [7]:
login(UserSecretsClient().get_secret("HUGGINGFACE_TOKEN"))

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
checkpoint = "CohereForAI/aya-23-8b"
quantization_config = None
if QUANTIZE_4BIT:
  quantization_config = BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_quant_type="nf4",
      bnb_4bit_use_double_quant=True,
      bnb_4bit_compute_dtype=torch.bfloat16,
  )
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
seq_lenght = 1500
tokenizer.model_max_length = seq_lenght
max_seq_length = seq_lenght
config = AutoConfig.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(
    checkpoint,
    config=config,
    quantization_config=quantization_config,
    torch_dtype="bfloat16",
    device_map=device,
    attn_implementation="flash_attention_2",
)

In [ ]:
silver_train_df = pd.read_csv("/kaggle/input/genai-ucu-2024-lab1-preprocessed/silver_train.csv")
silver_test_df = pd.read_csv("/kaggle/input/genai-ucu-2024-lab1-preprocessed/silver_test.csv")

In [21]:
example_template: str = """
Вхідний текст:
{example_text}
Знайдені сутності:
{example_labels}
"""

prompt_template = NERPrompt1().prompt_template.format(
    examples="".join([
        example_template.format(
            example_text=example["text"],
            example_labels=example["labels"]
        ) for example in THREE_SHOT_EXAMPLES_DICT
    ]),
    text="",
)
print(prompt_template)

Твоє завдання – виділити всі сутності у наданому тексті за наведеними категоріями та вивести їх у форматі JSON-списку:

Категорії сутностей:
- ART: артефакт (створений людиною предмет)
- DATE: дата (календарна дата, рік)
- DOC: документ (назви документів)
- JOB: посада (професійний титул, робоча позиція)
- LOG: місце (географічні об’єкти, назви країн, міст, річок тощо)
- MISC: різне (інші сутності, не підпадають під інші категорії)
- MON: гроші (сума, валюта)
- ORG: організація (установи, компанії, заклади)
- PCT: відсоток (число у відсотках)
- PERIOD: період (тривалість часу)
- PERS: особа (людські імена, прізвища)
- QUANT: кількість (числові значення)
- TIME: час (конкретний момент доби)

Формат відповіді: список об’єктів у JSON, кожен об’єкт має поля:
"label" – категорія сутності
"text" – фрагмент тексту сутності з оригінального тексту без змін

Нижче наведено приклади формату та стилю розпізнавання сутностей:

Вхідний текст:
український бізнес почав використовувати КСВ як інструмен

In [ ]:
for essay_id, essay_text in tqdm(feature_essay_dict.items(), desc="Essay progress"):
    inputs: list[str] = inputs_df.loc[inputs_df.loc[:, "essay_id"] == int(f_essay_id), "inputs"].to_list()
    inputs: list[str] = [prompt_template.format(original_text=input) for input in inputs]
    inputs: list[dict[str, str]] = get_message_format(inputs)
    input_ids = tokenizer.apply_chat_template(
        inputs,
        tokenize=True,
        add_generation_prompt=True,
        padding=True,
        return_tensors="pt",
    )
    input_ids = input_ids.to(model.device)
    prompt_padded_len = len(input_ids[0])
    # Generate corrections
    gen_tokens = model.generate(
        input_ids,
        temperature=parameters.baseline.temperature,
        top_p=parameters.baseline.top_p,
        top_k=parameters.baseline.top_k,
        max_new_tokens=seq_lenght,
        do_sample=True,
    )
    gen_tokens = [
        gt[prompt_padded_len:] for gt in gen_tokens
    ]
    outputs: list[str] = tokenizer.batch_decode(
        gen_tokens,
        skip_special_tokens=True
    )
    # Join divided texts if any
    correction = "".join(outputs)
    print(f_essay_id, correction)
    
    out_essays[f_essay_id] = "".join(correction)

    print("All essays corrected!")
    md_output = dict_to_md(out_essays)

In [ ]:
import json

def location_baseline(text):
    text = text[0].lower() + text[1:]
    text = text.split(" ")
    text = [item for item in text if item[0].isupper()]
    text = [{"label": "LOC", "text": item} for item in text]
    text = json.dumps(text)
    return text


In [ ]:
df["entities"] = df["text"].apply(location_baseline)
df = df.drop(columns=["text"])
df.to_csv("submission.csv", index=False)